# Lab 3: Build a Search Agent

In this lab, we'll use the Azure AI Agent Service to create an agent that is able to retrieve information from documents stored in Azure AI Search, a vector database. This pattern is known as retrieval augmented generation or RAG. The documents that we'll be searching are health insurance policies.

#### Step 1: Load packages

In [1]:
import os
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import AzureAISearchTool

load_dotenv() # Loads the environment variables and credentials we need to setup the agent

True

#### Step 2: Connect to your Azure AI Foundry project

In [2]:
# Connecting to our Azure AI Foundry project, which will allow us to use the deployed gpt-4o model
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["AIPROJECT_CONNECTION_STRING"],
    )

#### Step 3: Connect to your Azure AI Search index

In [3]:
# Iterate through the connections in your project and get the connection ID of the Aazure AI Search connection.
conn_list = project_client.connections.list()
conn_id = ""
for conn in conn_list:
    if conn.connection_type == "CognitiveSearch":
        conn_id = conn.id
        print(f"Connection ID: {conn.id}")

# Connect to your Azure AI Search index
ai_search = AzureAISearchTool(index_connection_id=conn_id, index_name="health-plan")

Connection ID: /subscriptions/654d52e1-f7cf-4b38-908f-79afaa1b1b48/resourceGroups/azure-ai-agents-1779832/providers/Microsoft.MachineLearningServices/workspaces/my-project-1779832/connections/mysearchservice1779832


#### Step 4: Define the search agent

In [4]:
search_agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="search-agent",
    instructions="You are a helpful agent that is an expert at searching health plan documents.",
    tools=ai_search.definitions,
    tool_resources=ai_search.resources,
) 

#### Step 5: Chat with the search agent

Let's test our search agent by asking it to give us information about the Northwind Standard health plan.

In [5]:
# The name of the health plan we want to search for
plan_name = 'Northwind Standard'

# Create a thread which is a conversation session between an agent and a user. 
thread = project_client.agents.create_thread()

# Create a message in the thread with the user asking for information about a specific health plan
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=f"Tell me about the {plan_name} plan.", # The user's message
)
# Run the agent to process tne message in the thread
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=search_agent.id)

# Check if the run was successful
if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Delete the agent when it's done running
project_client.agents.delete_agent(search_agent.id)

# Fetch all the messages from the thread
messages = project_client.agents.list_messages(thread_id=thread.id)

# Get the last message, which is the agent's resposne to the user's question
last_msg = messages.get_last_text_message_by_role("assistant")

# Display the agent's response
print('Agent:', last_msg.text.value)

Agent: The Northwind Standard plan is a comprehensive health plan that covers medical, vision, and dental services. It also includes preventive care services and prescription drug coverage. With this plan, you can access a variety of in-network providers, such as primary care physicians, specialists, hospitals, and pharmacies【3:0†source】【3:2†source】.

Northwind Standard covers several services, including:

- Preventive care services like physicals, immunizations, and screenings
- Diagnostic tests and treatments
- Medical consultations
- Physical therapy
- Mental health services
- Prescription drugs
- Inpatient hospital services
- Emergency services【3:1†source】.

However, the plan does not cover some types of services, such as cosmetic services, experimental treatments, and most dental services. It also does not cover services provided by out-of-network providers【3:1†source】【3:4†source】.

For costs, there are monthly premiums that must be paid to keep the coverage active, as well as pot